In [ ]:
import pandas as pd

df = pd.read_csv("dataset_mood_smartphone.csv")
df['time'] = pd.to_datetime(df['time'])
df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

df


Split the raw data into score and machine features

In [ ]:
score_variables = ["mood", "circumplex.arousal", "circumplex.valence", "activity"]

# Creating a dataset with only the selected variables
df_score = df[df['variable'].isin(score_variables)]

# Creating another dataset with the rest of the variables
df_machine = df[~df['variable'].isin(score_variables)]

df_score['date'] = df_score['time'].dt.date
df_machine['date'] = df_machine['time'].dt.date

# Displaying the first few rows of each dataset to verify
print("Dataset with score variables:")
print(df_score.head())
print("\nDataset with the machine data variables:")
print(df_machine.head())


Doing a "fake pivot" with a triple key so does not have to mind the duplicates

In [ ]:
df_score_piv = df_score.pivot_table(index=['index','id', 'date'], columns='variable', values='value', aggfunc='mean').reset_index()
df_machine_piv = df_machine.pivot_table(index=['index','id', 'date'], columns='variable', values='value', aggfunc='sum').reset_index()
df_score_piv.head(), df_machine_piv.head()
df_score_piv


In [ ]:
df_machine_piv


Aggregate after the pivot

In [ ]:
df_score_piv_agg = df_score_piv.groupby(['id', 'date']).mean().reset_index()
df_score_piv_agg


In [ ]:
df_machine_piv_agg = df_machine_piv.groupby(['id', 'date']).sum().reset_index()
df_machine_piv_agg

In [ ]:
# Assuming df_score_aggregated and df_machine_aggregated are your two tables
keys_score = set(df_score_piv_agg.apply(lambda row: (row['id'], row['date']), axis=1))
keys_machine = set(df_machine_piv_agg.apply(lambda row: (row['id'], row['date']), axis=1))

len(keys_score), len(keys_machine)


In [ ]:
missing_in_score = keys_machine - keys_score
missing_in_machine = keys_score - keys_machine

print("Missing in df_score_aggregated:", missing_in_score)
print("Missing in df_machine_aggregated:", missing_in_machine)


In [ ]:
# Merging with an outer join
df_merged = pd.merge(df_score_piv_agg, df_machine_piv_agg, on=['id', 'date'], how='outer', indicator=True)

# Filtering to see which rows are present in one table but not the other
missing_in_score_df = df_merged[df_merged['_merge'] == 'right_only']
missing_in_machine_df = df_merged[df_merged['_merge'] == 'left_only']

print("Rows missing in df_score_aggregated:")
print(len(missing_in_score_df))

print("\nRows missing #in df_machine_aggregated:")
print(len(missing_in_machine_df))
